# Maxwell's Equations (Unit 2.4)

[Peter Monk: "Finite Elements for Maxwell's Equations"]

## Magnetostatic field generated by a permanent magnet

magnetic flux $B$, magnetic field $H$, given magnetization $M$:

$$B=\mu(H+M), \hspace{1em} \text{div }B=0,\hspace{1em}\text{curl }H=0$$

Introducing a vector-potential $A$ such that $B=\text{curl}A$, and putting equations together we get (how?)
$$\text{curl }\mu^{−1} \text{curl }A=\text{curl }M$$

In weak form: Find $A\in H(curl)$ such that

$$\int \mu^{−1} \text{curl }A \text{ curl }v=\int M \text{ curl }v \ \ \forall v \in H(curl)$$

Usually, the permeability $\mu$
is given as $\mu=\mu_r \mu_0$, with $\mu_0=4\pi 10^{−7}$ the permeability of vacuum.

In [1]:
from ngsolve import *
from netgen.csg import *
import netgen.gui
%gui tk

Geometric model and meshing of a bar magnet:

In [2]:
box = OrthoBrick(Pnt(-3,-3,-3),Pnt(3,3,3)).bc("outer")
magnet = Cylinder(Pnt(-1,0,0),Pnt(1,0,0), 0.3) * OrthoBrick(Pnt(-1,-3,-3),Pnt(1,3,3))
air = box - magnet

geo = CSGeometry()
geo.Add (air.mat("air"))
geo.Add (magnet.mat("magnet").maxh(1))
# mesh = Mesh(geo.GenerateMesh(maxh=2)) # old way
geo.Draw()
mesh = Mesh(geo.GenerateMesh(maxh=2, curvaturesafety=1))
mesh.Curve(3)

In [3]:
mesh.GetMaterials(), mesh.GetBoundaries()

(('air', 'magnet'),
 ('outer',
  'outer',
  'outer',
  'outer',
  'outer',
  'outer',
  'default',
  'default',
  'default'))

- Define space, forms and preconditioner. To obtain a regular system matrix, we add a very small $L_2$ term
- We solve magnetostatics, so we can gauge by adding andarbitrary gradient field. A cheap possibility is to delete all basis-functions which are gradients (flag 'nograds')

### Notes:
Why is it considered an $L_2$ term?  Maybe because it's a mass term in an HCurl space, there are no continuity constraints?

In [5]:
fes = HCurl(mesh, order=3, dirichlet="outer", nograds=True)
print ("ndof =", fes.ndof)
u,v = fes.TnT()

from math import pi
mu0 = 4*pi*1e-7
mur = CoefficientFunction( [1000 if mat== "magnet" else 1 
                            for mat in mesh.GetMaterials()]) 

a = BilinearForm(fes)
a += SymbolicBFI(1/(mu0*mur) * curl(u) * curl(v) + 1e-8/(mu0*mur)*u*v)
c = Preconditioner(a, "bddc")

f = LinearForm(fes)
mag = CoefficientFunction((1,0,0)) * \
    CoefficientFunction( [1 if mat == "magnet" else 0 for mat in mesh.GetMaterials()])
f += SymbolicLFI(mag*curl(v), definedon=mesh.Materials("magnet"))

ndof = 21439


Assemble system and setup preconditioner using (shared memory) task-parallelization:

In [6]:
with TaskManager():
    a.Assemble()
    f.Assemble()

In [8]:
gfu = GridFunction(fes)
with TaskManager():
    solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=c.mat)

#inv = CGSolver(a.mat, c.mat) (old way)
#gfu.vec.data = inv * f.vec


it =  0  err =  0.004834556218517531
it =  1  err =  0.0028938795257507656
it =  2  err =  0.0018764053599183437
it =  3  err =  0.001216702442057365
it =  4  err =  0.0008154078108549021
it =  5  err =  0.0005512980608231896
it =  6  err =  0.0003783687160592129
it =  7  err =  0.00024850704793478843
it =  8  err =  0.0001475159193971676
it =  9  err =  0.00010046059754749752
it =  10  err =  6.0599386314055175e-05
it =  11  err =  3.730674670588009e-05
it =  12  err =  2.2865488299441777e-05
it =  13  err =  1.3402569891395715e-05
it =  14  err =  8.744091600381492e-06
it =  15  err =  5.593363696871147e-06
it =  16  err =  3.19219394004555e-06
it =  17  err =  1.911582069014548e-06
it =  18  err =  1.1537923932285419e-06
it =  19  err =  6.662888413160118e-07
it =  20  err =  4.1644321790543207e-07
it =  21  err =  2.718353485497309e-07
it =  22  err =  1.6639462225488448e-07
it =  23  err =  1.0231846511415764e-07
it =  24  err =  5.8948887431628926e-08
it =  25  err =  3.554213427

In [9]:
# Draw (curl(gfu), mesh, "B-field") (old)

# the vector potential is not supposed to look nice
Draw (gfu, mesh, "vector-potential", draw_surf=False)

Draw (curl(gfu), mesh, "B-field", draw_surf=False)
Draw (1/(mu0*mur)*curl(gfu)-mag, mesh, "H-field", draw_surf=False)